<a href="https://colab.research.google.com/github/wesley28/ProcessamentoDigitalDeImagens/blob/main/Fatiamento_bits_att15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import math
from collections import Counter

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set_theme(style="darkgrid")


class ImagemBPM:

  def __init__(self, tipo='', largura=0, altura=0, valor_maximo = 1, colorido=False):
    self.tipo = tipo
    self.largura = largura
    self.altura = altura
    self.valor_maximo = valor_maximo
    self.colorido = colorido


  def _read(self, path):
    return open(path, "r").read().split('\n')


  def extrair_imagem(self, path, default_rgb = 0):
    ls_imagem = self._read(path)

    if self.colorido:
      self.tipo = ls_imagem[0]
      self.largura = int(ls_imagem[2].split()[0])
      self.altura = int(ls_imagem[2].split()[1])
      self.valor_maximo = int(ls_imagem[3])

      conteudo = ls_imagem[5:]
      conteudo = [i.split(' ') for i in conteudo]
      np_conteudo = np.array(conteudo)
      np_conteudo = np.concatenate(np_conteudo)
      np_conteudo = np.delete(np_conteudo, np.where(np_conteudo == ''))
      self.conteudo = [int(i) for i in np_conteudo]

      lista_rgb = {
          'red': np.array([]),
          'green': np.array([]),
          'blue': np.array([])
      }

      lista_rgb['red'] = np.array([self.conteudo[i] if i % 3 == 0 else default_rgb for i in range(len(self.conteudo))])
      lista_rgb['green'] = np.array([self.conteudo[i] if i % 3 == 1 else default_rgb for i in range(len(self.conteudo))])
      lista_rgb['blue'] = np.array([self.conteudo[i] if i % 3 == 2 else default_rgb for i in range(len(self.conteudo))])
      self.lista_rgb = lista_rgb

      lista_rgb_hist = {
          'red': np.array([]),
          'green': np.array([]),
          'blue': np.array([])
      }

      lista_rgb_hist['red'] = np.array([self.conteudo[i] for i in range(len(self.conteudo)) if i % 3 == 0])
      lista_rgb_hist['green'] = np.array([self.conteudo[i] for i in range(len(self.conteudo)) if i % 3 == 1]) # 1:0 4:1 7:2 10:3   1 3 5 7     i div 3
      lista_rgb_hist['blue'] = np.array([self.conteudo[i] for i in range(len(self.conteudo)) if i % 3 == 2])  # 2:0 5:1 8:2 11:3   1 3 5 7     i div 3
      self.lista_rgb_hist = lista_rgb_hist
      self.conteudo = np.array(self.conteudo)

    else:
      self.tipo = ls_imagem[0]
      self.largura = int(ls_imagem[1].split()[0])
      self.altura = int(ls_imagem[1].split()[1])
      self.valor_maximo = int(ls_imagem[2])
      self.conteudo = np.array([int(i) if i != '' else 0 for i in ls_imagem[2 : self.altura * self.largura + 2]])


  def fatiamento_bits(self):
    cont = self.conteudo
    bin_cont = [np.binary_repr(c, width=8) for c in cont]

    dt_lists_bin_cont = {}
    for i in range(8):
      dt_lists_bin_cont[i] = np.array([int(bin[i]) for bin in bin_cont])

    self.dt_lists_bin_cont = dt_lists_bin_cont


  def bits_significativos(self):
    list_bits_significativos = []
    for i in range(len(self.dt_lists_bin_cont[0])):
      list_bits_significativos.append(
          str(self.dt_lists_bin_cont[0][i]) + str(self.dt_lists_bin_cont[1][i]) + str(self.dt_lists_bin_cont[2][i]) + '0' * 5)

    self.list_bits_significativos = np.array(list_bits_significativos)


  def gerar_hist(self, nome_arquivo, rgb=None):
    if rgb is None:
      self._hist(pd.Series(self.conteudo), nome_arquivo)

    else:
      self._hist(pd.Series(self.lista_rgb_hist[rgb]), nome_arquivo+'_'+rgb, rgb)


  def _hist(self, df, nome_arquivo, cor='gray'):
    ax = df.hist(color = cor)
    fig = ax.get_figure()
    fig.savefig(nome_arquivo+'.jpg')


  def transformacao_linear(self, escala_cinza=True):

    if escala_cinza:
      self.conteudo_tranformacao_linear = self._transformacao_linear(self.conteudo)

    else:
      conteudo_tranformacao_linear_red = self._transformacao_linear(self.lista_rgb_hist['red'])
      conteudo_tranformacao_linear_green = self._transformacao_linear(self.lista_rgb_hist['green'])
      conteudo_tranformacao_linear_blue = self._transformacao_linear(self.lista_rgb_hist['blue'])

      lt_conteudo_rgb_transformacao_linear = []
      for i in range(len(conteudo_tranformacao_linear_red) * 3):
        if i % 3 == 1:
          lt_conteudo_rgb_transformacao_linear.append(conteudo_tranformacao_linear_green[i // 3])
        elif i % 3 == 2:
          lt_conteudo_rgb_transformacao_linear.append(conteudo_tranformacao_linear_blue[i // 3])
        else:
          lt_conteudo_rgb_transformacao_linear.append(conteudo_tranformacao_linear_red[i // 3])
      self.conteudo_tranformacao_linear = np.array(lt_conteudo_rgb_transformacao_linear)



  def _transformacao_linear(self, valores):
    min_valor = min(valores)
    max_valor = max(valores)
    self.a = 255 / (max_valor - min_valor) # Y = x * a + b
    self.b = -self.a * min_valor

    lt_conteudo_tranformacao_linear = []
    for vlr in self._gen_transformacao_linear(valores):
      lt_conteudo_tranformacao_linear.append(vlr)
    return np.array(lt_conteudo_tranformacao_linear)


  def _gen_transformacao_linear(self, valores):
    for vlr in valores:
      yield round(self.a * vlr + self.b)


  def compactar(self, fator):
    assert self.altura % fator == 0, 'O fator de compactação precisa ser múltiplo da altura'
    assert self.largura % fator == 0, 'O fator de compactação precisa ser múltiplo da largura'

    self.fator = fator
    self.nova_altura = int(self.altura / self.fator)
    self.nova_largura = int(self.largura / self.fator)

    rs_conteudo = self.conteudo.reshape(int((self.altura * self.largura) / (self.fator * self.fator)), self.fator * self.fator)

    conteudo_compactado = []
    for agrupamento in self._gen_agrupamento_pixels(rs_conteudo):
      conteudo_compactado.append(agrupamento)

    self.conteudo_compactado = conteudo_compactado


  def gerar_media_pixels_rgb(self):
    self.gerar_media_pixels()
    conteudo_media_rgb = np.array(range(self.conteudo_agrupado.shape[0] * 3))
    for i in range(len(self.conteudo_agrupado)):
      conteudo_media_rgb[3 * i: 3 * i + 3] = self.conteudo_agrupado[i]

    self.conteudo_media_rgb = conteudo_media_rgb


  def gerar_media_pixels(self, escala_cinza=False):
    np_array = np.array(self.conteudo)
    np_array_reshape = np_array.reshape(np_array.shape[0] // 3, 3)
    conteudo_agrupado = []
    for agrupamento in self._gen_agrupamento_pixels(np_array_reshape):
      conteudo_agrupado.append(agrupamento)

    self.conteudo_agrupado = np.array(conteudo_agrupado)

    if escala_cinza:
      self.tipo = 'P2'


  def _gen_agrupamento_pixels(self, ls_conteudo):
    for valores in ls_conteudo:
      #yield max(valores)
      #yield min(valores)
      yield round(sum(valores) / len(valores))
      #c = Counter(valores)
      #yield c.most_common(1)[0][0]


  def redimensionar(self, nv_largura, nv_altura):
    self.nv_largura = nv_largura
    self.nv_altura = nv_altura
    matriz_conteudo = self.conteudo.reshape(self.largura, self.altura)
    matriz_conteudo_copia = matriz_conteudo.copy()

    diff_largura = nv_largura - self.largura
    if diff_largura != 0:
      lt_linhas = list(range(self.largura))
      random.shuffle(lt_linhas)

      if diff_largura > 0:
        linhas_add = lt_linhas[:diff_largura]
        for id in linhas_add:
          matriz_conteudo_copia = np.insert(matriz_conteudo_copia, self.largura * id, matriz_conteudo[id, :])

      else:
        linhas_rmv = lt_linhas[diff_largura:]
        index_rmv = []
        for row in linhas_rmv:
          index_rmv.append([i for i in range(row * self.largura, (row + 1) * self.largura)])
        index_rmv = list(np.concatenate(index_rmv).flat)
        matriz_conteudo_copia = np.delete(matriz_conteudo_copia, index_rmv)

      matriz_conteudo_copia = matriz_conteudo_copia.reshape(nv_largura, self.altura)
      self.conteudo_redimensionado = matriz_conteudo_copia.flatten()

    matriz_conteudo_copia2 = matriz_conteudo_copia.copy()
    diff_altura = nv_altura - self.altura
    if diff_altura != 0:
      lt_colunas = list(range(self.altura))
      random.shuffle(lt_colunas)
      if diff_altura > 0:
        colunas_add = lt_colunas[:diff_altura]

        for id in colunas_add:
          matriz_conteudo_copia2 = np.insert(matriz_conteudo_copia2, self.altura * id, matriz_conteudo_copia[:, id])

        self.conteudo_redimensionado = matriz_conteudo_copia2

      else:
        colunas_rmv = lt_colunas[diff_altura:]
        index_rmv = []
        for col in colunas_rmv:
          index_rmv.append([i for i in range(col * self.nv_largura, (col + 1) * self.nv_largura)])
        index_rmv = list(np.concatenate(index_rmv).flat)
        matriz_conteudo_copia2 = np.delete(matriz_conteudo_copia2, index_rmv)

      self.conteudo_redimensionado = matriz_conteudo_copia2.flatten()



  def _gerar_cabecalho(self):
    return '{}\n{} {}\n {}\n'.format(self.tipo, self.largura, self.altura, self.valor_maximo)


  def salvar_imagem(self, nome, formato):
    with open('{}.{}'.format(nome, formato), 'w') as fd:
      fd.write(self.imagem)


  def salvar_imagem_fatiamento_bits(self, nome, formato):
    for i in range(8):
      with open('{}.{}'.format(nome + "_" + str(i), formato), 'w') as fd:
        fd.write(self.imagens[i])


  def salvar_imagem_rgb(self, nome, formato):
    with open('{}.{}'.format(nome+'_red', formato), 'w') as fd:
      fd.write(self.imagem_red)

    with open('{}.{}'.format(nome+'_green', formato), 'w') as fd:
      fd.write(self.imagem_green)

    with open('{}.{}'.format(nome+'_blue', formato), 'w') as fd:
      fd.write(self.imagem_blue)


  def gerar_imagem(
      self, colorido = False, random = True, compactado = False, redimensionar = False, gerar_rgb = False, media_escala_cinza = False,
      media_rgb=False, transformacao_linear=False, fatiamento_bits=False):

    if random:
      cabecalho = self._gerar_cabecalho()
      matriz = '\n'.join([' '.join([str(random.randint(0, self.valor_maximo)) for _ in range(self.altura * 3 if colorido else self.altura)]) for _ in range(self.largura)])
      self.imagem = cabecalho + matriz

    elif compactado:
      self.imagem = '{} \n{} {} \n{} \n{}'.format(
          self.tipo, self.nova_largura, self.nova_altura, self.valor_maximo,
          '\n'.join([str(i) + ' ' for i in self.conteudo_compactado]))

    elif redimensionar:
      self.imagem = '{}\n{} {}\n{}'.format(
          self.tipo, self.nv_largura, self.nv_altura,
          '\n'.join([str(i) for i in self.conteudo_redimensionado.tolist()]))

    elif gerar_rgb:
      self.imagem_red = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.lista_rgb['red'].tolist()]))
      self.imagem_green = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.lista_rgb['green'].tolist()]))
      self.imagem_blue = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.lista_rgb['blue'].tolist()]))

    elif media_escala_cinza:
      self.imagem = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.conteudo_agrupado.tolist()]))

    elif media_rgb:
      self.imagem = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.conteudo_media_rgb.tolist()]))

    elif transformacao_linear:
      self.imagem = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.conteudo_tranformacao_linear.tolist()]))

    elif fatiamento_bits:
      imagens = {}
      for i in range(8):
        imagens[i] = '{}\n{} {}\n{}\n{}'.format(
            self.tipo, self.largura, self.altura, 1,
            '\n'.join([str(i) for i in self.dt_lists_bin_cont[i].tolist()]))
        self.imagens = imagens

    else:
      self.imagem = '{}\n{} {}\n{}\n{}'.format(
          self.tipo, self.largura, self.altura, self.valor_maximo,
          '\n'.join([str(i) for i in self.conteudo.tolist()]))


In [ ]:
t1 = ImagemBPM(colorido=True)
t1.extrair_imagem("/content/Fig0314(a)(100-dollars).txt")
t1.fatiamento_bits()
t1.gerar_imagem(random=False, compactado=False, redimensionar=False, fatiamento_bits=True)
t1.salvar_imagem_fatiamento_bits('fatiamento_bits', '')

In [ ]:
t1.bits_significativos()
t1.gerar_imagem(random=False, compactado=False, redimensionar=False)
t1.salvar_imagem('fatiamento_bits_significativos', 'pgm')